In [1]:
import numpy as np
import matplotlib.pyplot as plt
import skimage.morphology as morph
%matplotlib inline

# Exercise 1 : basic operations
Test the effect of erosion, dilation, open, close on the test
image sand_bilevel.png. 

Change size and type of the structure elements.

## Erode

## Dilate

## Open

## Close

# Exercise 2 : Illumination correction
Find a structure element that corrects the inhomogeneous illumination in the image sand_grad.png using top-hats. 

Is there a difference in SE size when you use black top-hat or white top-hat, why?

# Exercise 3 : Distance maps
Compute distance maps of the grains in the sand image using the
three metrics Euclidean, City-block, and Chessboard. Compute difference images between the first one
and the other two. What conclusion can you make regarding the
accuracy of the two connectivity based metrics compared to the
Euclidean metric.

Keep the Euclidean distance map for the next exercises.

# Exercise 4: Local extrema

Compute the distance maps of both grains and pore space in the
sand image and combine them into one image as
\[
g=D_{\mathcal{E}}(f^c)-D_{\mathcal{E}}(f)
\]
Test the palette of local extrema functions on $g$.

# Exercise 5: Watershed segmentation
Reading: http://scikit-image.org/docs/dev/api/skimage.morphology.html?highlight=watershed#skimage.morphology.watershed

To segment a binary image with WS you have to compute it's
distance map (hopefully already done\ldots). The distance you have
does not show valleys as we want but rather mountains, hence we
have to negate the distance map:

\[
D=-D
\]

We must also define the pore space as not allowed

$D(~sand)=-inf$

which means that all pixels belonging to the complement of the
grains, i.e. the pore-space, are assigned the value $-\infty$.
After these steps is the image ready to segment
$L=watershed(D)$
